In [1]:
import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"

In [2]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
from numba import njit, types, typed
from numba.experimental import jitclass
import logging
from nb_quantfreedom.nb_base import final_bt_df_only
from nb_quantfreedom.nb_enums import *

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
candles_np = mufex_main.get_candles_df(symbol="BTCUSDT", timeframe="1m", candles_to_dl=300, limit=200)

It took 00 mins and 00 seconds to download 300 candles


In [4]:
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)

In [5]:
(
    checker_sl_hit,
    checker_sl_to_be,
    checker_tsl,
    dec_pos_calculator,
    log_func_list,
    str_func_list,
    sl_bcb_price_getter,
    sl_calculator,
    sl_mover,
    zero_or_entry_calc,
) = final_bt_df_only(
    logger_type=LoggerType.Print,
    static_os=static_os,
)

In [6]:
(
    account_state,
    fees_paid,
    order_result,
    realized_pnl,
) = dec_pos_calculator(
    average_entry=23456.0,
    equity=1000.0,
    exit_fee_pct=0.0003,
    exit_price=23684.6,
    logger=log_func_list,
    stringer=str_func_list,
    market_fee_pct=0.0009,
    position_size_asset=0.02,
)

[cache] index saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_helper_funcs.get_n_digits-131.py310.nbi'
[cache] data saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_helper_funcs.get_n_digits-131.py310.1.nbc'
[cache] index saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_helper_funcs.float_to_str-148.py310.nbi'
[cache] data saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_helper_funcs.float_to_str-148.py310.1.nbc'
[cache] index saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.print_float_to_str-204.py310.nbi'
[cache] data saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.print_float_to_str-204.py310.1.nbc'
[cache] index saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.print_log_debug-184.py310.nbi'
[cache] data saved t

TypeError: can't unbox heterogeneous list: type(CPUDispatcher(<function print_log_debug at 0x00000133977E8310>)) != type(CPUDispatcher(<function print_log_info at 0x00000133977E8160>))

In [ ]:
@njit(cache=True)
def test_dec(dec_pos, log_list, str_list):
    (
        account_state,
        fees_paid,
        order_result,
        realized_pnl,
    ) = dec_pos(
        average_entry=23456.0,
        equity=1000.0,
        exit_fee_pct=0.0003,
        exit_price=23684.6,
        logger=log_list,
        stringer=str_list,
        market_fee_pct=0.0009,
        position_size_asset=0.02,
    )
    x = 10 + fees_paid
    return x

In [ ]:
test_dec(dec_pos_calculator, log_func_list, str_func_list)

In [ ]:
from numba import typed, typeof, njit, int64, types
from numba.experimental import jitclass
from numba.extending import overload_method, overload

spec = [
    ('value', int64),
]


@jitclass(spec)
class Bag(object):
    def __init__(self, value):
        self.value = value

mybag = Bag(21)

x = typed.List.empty_list(Bag.class_type.instance_type)
x.append(mybag)

# @overload_method(types.misc.ClassInstanceType, 'clone')
# def ol_bag_clone(inst,):
#     if inst is Bag.class_type.instance_type:
#         def impl(inst,):
#             return Bag(inst.value)
#         return impl

@njit
def foo(z):
    t = z[0]
    t.clone()

foo(x)